# Data Preprocessing and Splitting
---

In [1]:
# import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [2]:
# load data

bbox_df = pd.read_pickle('../data/interim/bbox_df.pkl')
data_entry_df = pd.read_pickle('../data/interim/data_entry_df.pkl')
train_val_list = pd.read_pickle('../data/interim/train_val_list.pkl')
test_list = pd.read_pickle('../data/interim/test_list.pkl')

In [3]:
# split multi-labeled data up

"""
labels_exploded = data_entry_df["finding_labels"].str.split('|').explode()
len(labels_exploded.unique())
"""

'\nlabels_exploded = data_entry_df["finding_labels"].str.split(\'|\').explode()\nlen(labels_exploded.unique())\n'

In [4]:
# [add folder numbers to the dataset]

"""

# extracted the targeted images
targeted_imgs = list(data_entry_df["image_index"])

# extract only the images folders paths from kaggle and store it in a list
folder_paths = []

for folder in os.listdir(path):

    if folder.startswith("images") == True:
        f_path = os.path.join(path, folder, "images")
        folder_paths.append(f_path)


# search folders based on targeted images
parent_folders = []

for img in targeted_imgs:
    found = False
    for folder in folder_paths:
        if img in os.listdir(folder):
            parent_folders.append(os.path.basename(os.path.dirname(folder)))
            found = True
            break  
    if not found:
        print(f"Not found: {img}")

# create a dataframe 
parent_folders_df = pd.DataFrame({
    "image_index": targeted_imgs,
    "folders": parent_folders
})



"""

'\n\n# extracted the targeted images\ntargeted_imgs = list(data_entry_df["image_index"])\n\n# extract only the images folders paths from kaggle and store it in a list\nfolder_paths = []\n\nfor folder in os.listdir(path):\n\n    if folder.startswith("images") == True:\n        f_path = os.path.join(path, folder, "images")\n        folder_paths.append(f_path)\n\n\n# search folders based on targeted images\nparent_folders = []\n\nfor img in targeted_imgs:\n    found = False\n    for folder in folder_paths:\n        if img in os.listdir(folder):\n            parent_folders.append(os.path.basename(os.path.dirname(folder)))\n            found = True\n            break  \n    if not found:\n        print(f"Not found: {img}")\n\n# create a dataframe \nparent_folders_df = pd.DataFrame({\n    "image_index": targeted_imgs,\n    "folders": parent_folders\n})\n\n\n\n'

In [5]:
# note: during data splitting, do an 80/20 split based on the data listed in the train_val_list file. use the test_list file for model evaluation